# Python Script Cleaner for CSV_files CapStone

In [1]:
import pandas as pd
import random
import os
import numpy as np

In [9]:
#Directory that Contains CSV, Subsequent files will be generated into here
csv_directory = (r"C:\Users\micha\OneDrive\Desktop\BE177\FULL_DATA_CLEANUP2")

#MAIN Data File you wish to clean
file_name = (r"OGDATA.csv")
csv_df = pd.read_csv(os.path.join("r",csv_directory,file_name))

#First lets drop Entries (Rows) that have NaN or NA values
nan_filter = csv_df.dropna()
nan_filter = nan_filter.sort_values(by='title') #Want to alphabatize them
nan_filter['resolution'] = pd.to_numeric(nan_filter['resolution'])

#Lets Save this Starting File with all the OG DATA
nan_filter.to_csv(os.path.join("r",csv_directory,"OGDATA.csv"), index = False)

#Lets save the length of this dataframe, so we know hm was removed from this OG
OG_Length = len(nan_filter)

## Soft Pass Filter

One of the columns xref_UNIPROTKB keeps track of the ways each Cryo-EM has been referenced. If two entries have the same exact references they will be the same exact protein therefore remove this repeated entry.

In [10]:
##Need to filter out similar proteins
#Return Type is a series (array) which contains True and False for rows you want to keep or naw depending on last argument
non_unique_mask = nan_filter.duplicated(subset='xref_UNIPROTKB', keep=False)
non_unique_df = nan_filter[non_unique_mask] #Gives you a dataframe that has all the duplicates
unique_df = nan_filter.drop(non_unique_df.index) #Drops the rows that are not unique based on previous dataframe

df = unique_df
HP_Length = len(unique_df)
df.to_csv(os.path.join(csv_directory))

file_name = file_name+"_HARDPASS.csv"
df.to_csv(os.path.join(csv_directory,file_name), index= False) #Lets Save this harpass filtered stuff
non_unique_df
print("% Removed during Soft Pass: ",round(((HP_Length-OG_Length)/OG_Length)*100),"%")

% Removed during Soft Pass:  -52 %


#### Items caught in the softpass dataframe, will now be cleaned to ensure for entries of the same protein only the entry with the lowest resoloution is kept!

In [11]:
non_unique_df
lst = []

#Sort by groups of xref_UNIPROTKB
grouped_proteins = non_unique_df.groupby("xref_UNIPROTKB")
for ref, group in grouped_proteins:
    group = group.dropna()
    high_res = group.loc[group["resolution"].idxmin()]
    lst.append(high_res) #Only append highest-res entries   
    
#Keep the highest res shit
high_res_df = pd.DataFrame(lst,columns = non_unique_df.columns)
kept_highRes = len(non_unique_df) - len(high_res_df)

#Save it
high_res_df.to_csv(os.path.join("r",csv_directory,"Post_HARDPASS_FILTERED_HIGHRES.csv"))
#print("% Removed From SoftPass: ",round((((HP_Length+kept_highRes)-OG_Length)/OG_Length)*100,2),"%") ##For Testing purposes

% Removed From SoftPass:  -19.46 %


#### Now go back and readd the filtered entries to the dataframe with unique entries, thus all entries contain unique proteins!

In [5]:
##Join 2 dataframes
result = pd.concat([unique_df, high_res_df], ignore_index=True)
result = result.sort_values(by='title')
result.to_csv(os.path.join("r",csv_directory,"RAW_DATA.csv"))
df=result
#OG_SP = len(result) ##For testing purposes

## HardPass Filter

While proteins with the exact same references are filtered out, there exists proteins that have overlapping references. In this case the proteins are not unique from each other thus we need to filter them out.

In [6]:
# Function to check if there's at least one common sequence
def has_common_sequence(row1, row2):
    sequences1 = set(row1["xref_UNIPROTKB"].split(','))  # Split and convert to a set for efficient comparison
    sequences2 = set(row2["xref_UNIPROTKB"].split(','))
    return bool(sequences1 & sequences2)  # Check for common elements using set intersection


In [7]:
dict_group = {'class':[0]*len(df)}
#df_flag = pd.DataFrame(dict_flag)
df_group = pd.DataFrame(dict_group)
df = pd.concat([df,df_group], ignore_index=False, axis = 1)

In [8]:
grouped_dfs = {}
indexes_dropped = []
xRef = []
unique = []
for i in range(len(df)):
    if i not in indexes_dropped:
        xRef = []
        key = f'DataFrame_{i + 1}' # Create a key for each DataFrame
        hasSeenI = False
        for j in range(i+1,len(df)):
            if has_common_sequence(df.iloc[i], df.iloc[j]):
                hasSeenI = True
                xRef.append(j) #Type NP array
                indexes_dropped.append(j)
                
        if (hasSeenI):
            xRef.append(i)
            #print(xRef)
            indexes_dropped.append(i)
            #xRefDf_temp = pd.DataFrame(xRefGrp,columns = df.columns)
            #xRefDf = xRefDf.concat([xRefDf,xRefDf_temp], axis = 1)
            grouped_dfs[key] = xRef
        else:
            unique.append(i) #If it was a unique row

# print()
# print(len(grouped_dfs))
# print(len(unique))
# print(df.shape[0])

[4, 7, 8, 9, 17, 23, 24, 25, 26, 27, 30, 31, 32, 35, 36, 37, 38, 39, 63, 75, 76, 77, 78, 79, 89, 90, 91, 92, 93, 94, 95, 96, 118, 119, 120, 121, 133, 134, 135, 203, 204, 213, 215, 216, 217, 218, 259, 260, 279, 280, 281, 322, 323, 324, 325, 359, 360, 361, 362, 365, 369, 370, 371, 372, 374, 375, 376, 377, 378, 379, 380, 381, 382, 386, 391, 392, 393, 394, 395, 397, 400, 401, 408, 413, 490, 493, 499, 521, 524, 526, 547, 550, 559, 565, 566, 569, 570, 604, 605, 616, 617, 618, 619, 670, 673, 677, 680, 681, 684, 685, 690, 691, 693, 694, 695, 700, 714, 717, 728, 730, 743, 744, 745, 746, 747, 752, 754, 755, 756, 758, 788, 789, 808, 837, 838, 851, 852, 878, 893, 899, 900, 903, 904, 0]
[36, 37, 730, 1]
[16, 398, 399, 654, 655, 656, 738, 2]
[10, 19, 67, 68, 69, 241, 242, 301, 302, 312, 313, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 491, 492, 658, 659, 660, 705, 706, 707, 709, 723, 727, 772, 773, 781, 782, 3]
[40, 83, 100, 130, 138, 139, 140, 141, 142, 143, 144, 145, 146,

In [9]:
#Go Through each group in dataframe and assign same class
for i, key in enumerate(grouped_dfs):
    for j in grouped_dfs[key]:
        df.at[j, 'class'] = i

In [9]:
kept_indexes = []
non_unique = []
for i, key in enumerate(grouped_dfs):
    kept_indexes.append(random.choice(grouped_dfs[key]))
    non_unique = non_unique + grouped_dfs[key]


unique = unique + kept_indexes
final_df = df.iloc[unique].reset_index(drop=True)
similar_proteins = df.iloc[non_unique].reset_index(drop=True)

    
    

In [10]:
similar_proteins = similar_proteins.sort_values(by='class')
similar_proteins.to_csv(os.path.join("r",csv_directory,"similar_proteins.csv"), index= False) #Lets Save this harpass filtered stuff
final_df.to_csv(os.path.join(csv_directory,"FINAL_filtered.csv"),index=False)

## RUN this code on the final filtered file to obtain statistics automatically, or dont its annoying to count.

In [4]:
# csv_file = (r"C:\Users\micha\OneDrive\Desktop\BE177\FULL_DATA_CLEANUP\Proteins_Caught_In_Filter\Post_HARDPASS_FILTERED_HIGHRES.csv")
# csv_directory = r"C:\Users\micha\OneDrive\Desktop\BE177\FULL_DATA_CLEANUP\Proteins_Caught_In_Filter"
# final_df_groups = pd.read_csv(csv_file)
# final_df_groups = final_df_groups.groupby(final_df_groups['Type'])
# Protein_Types = []
# Protein_Type_Counts = []
# for group in final_df_groups:
#     Protein_Types.append(group[0])
#     Protein_Type_Counts.append(group[1].shape[0])

# final_df_groups = {key : value for key, value in zip(Protein_Types, Protein_Type_Counts)}
# final_df_groupz = pd.DataFrame.from_dict([final_df_groups])
# final_df_groupz.to_csv(os.path.join(csv_directory,"InitialCounts.csv"),index=False)